# 개체명 인식 (Named Entity Recognition)

개체명 인식이란 문장 내에 포함된 어떤 단어가 인물, 장소, 날짜 등을 의미하는 단어인지 인식하는 것이다. 딥러닝 모델이나 확률 모델 등을 이용해 문장에서 개체명을 인식하는 프로그램을 개체명 인식기라 부른다. 예를 들어 날짜와 지역에 대해 개체 인식을 할 수 있는 개체명 인식 모델이 있다고 했을 때, 다음과 같이 문장을 분류한다.

-  1. 입력 문장 : 내일 파리 날씨 알려줘
-  2. 문장 의도 분류 : 날씨 요청
-  3. 개체명 인식 결과 : <br>
  내일 - 날짜 <br>
  파리 - 지역

개체명 인식 모델을 만들기 위해서는 우선 BIO 표기법을 알아야 한다. BIO란, Beginning, Inside, Outside의 약자로 각 토큰마다 태그를 붙이기 위해 사용한다. B (beginning)는 개체명이 시작되는 단어에 'B-개체명'으로 태그 되며, I(inside)는 'B-개체명'과 연결되는 단어일 때 'I-개체명'으로 태그된다. 마지막으로 O(outside)는 개체명 이외의 모든 토큰에 태그된다. 

오늘부터 사쿠라이 쇼는 게이오 대학교에 등교합니다.

오늘 / B-Date, 사쿠라이 / B-Person, 쇼 / I-Person, 게이오 / B-University, 대학교 / I-University, 근무 / o, 부터 / o, 는 / o, 에 / o, 합니다. / o   

여기 예제에서 사용하는 BIO 태그 학습 데이터셋은 HLCT 2016에서 제공하는 말뭉치 데이터를 수정한 KoreanNERCorpus입니다.<br>
[github.com/machingreading/KoreanNERTCorpus](https://github.com/machingreading/KoreanNERTCorpus)

train.txt 파일 내용을 살펴보면, ;으로 시작하는 문장 라인은 원본 문장에 해당하며, $로 시작하는 문장 라인은 해당 문장에서 NER 처리된 결과를 의미한다.
그다음 라인부터는  토큰 번호, 단어 토큰, 품사 태그, BIO 태그로 구성된 열이 존재한다. 여기서는 단어 토큰과 BIO 태그 정보만 학습 데이터셋으로 사용한다.

구현하는 개체명 인식기의 원리는 다음과 같다. 해당 모델은 단어 토큰을 입력했을 때 출력되는 NER 태그값을 예측하는 문제다. 예를 들어 '삼성전자'를 입력했을 때 단체를 뜻하는 B_OG(oraganization) 태그가 출력되도록 모델을 학습한다.

In [10]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

학습 데이터 구조에 맞게 파일을 읽어와 문장 라인별로 토큰 번호, 단어 토크, 품사 태그, BIO 태그 정보를 불러온다.

In [15]:
# 학습 파일 불러오기
def read_file(file_name):
  sents = []
  with open(file_name, 'r', encoding= 'utf-8') as f:
    lines = f.readlines()
    for idx, l in enumerate(lines):
      if l[0]== ';' and lines[idx+1][0] == '$':
        this_sent = []
      elif l[0] == '$' and lines[idx -1][0] == ';':
        continue
      elif l[0] == '\n':
        sents.append(this_sent)
      else :
        this_sent.append(tuple(l.split()))
  return sents

In [16]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('train.txt')

In [19]:
# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus: 
  tagged_sentence = []
  sentence, bio_tag = [],[]
  for w in t :
    tagged_sentence.append((w[1],w[3]))
    sentence.append(w[1])
    bio_tag.append(w[3])

  sentences.append(sentence)
  tags.append(bio_tag)

0번째 원본 문장에서 분리된 단어 토큰들이 sentences 리스트에 저장된다. 저장된 단어 시퀀스는 다음과 같은데, 이 단어 시퀀스에 해당하는 BIO 태그 정보들이 tags 리스트에 저장된다. 이때 sentences 리스트와 tags 리스트의 크기는 동일하다. 단어 시퀀스의 평균 길이값을 기준으로 시퀀스 패딩의 크기를 결정한다.

In [34]:
print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302


단어 시퀀스와 태그 시퀀스를 사전으로 만들기 위해 토크나이저를 정의한 후 fit_on_texts() 함수를 호출한다. 여기서 OOV는 out of vocabulary의 약자로 단어 사전에 포함되지 않은 단어를 의미한다. 단어 사전의 첫번째 인덱스 토큰값으로 'OOV'를 설정한다.

In [35]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token= 'OOV') # 첫 번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower= False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

생성된 사전 리스트를 이용해 단어와 태그 사전의 크기를 정의한다.

In [41]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index)+1
tag_size = len(tag_tokenizer.word_index)+1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 : ", vocab_size)

BIO 태그 사전 크기 : 8
단어 사전 크기 :  13834


위에서 만들어진 사전 데이터를 시퀀스 번호 형태로 인코딩한다.

In [42]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [43]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0]= 'PAD'

개체명 인식 모델의 입출력 벡터 크기를 동일하게 맞추기 위해 시퀀스 패딩 작업을 한다. 벡터 크기를 위에서 계산한 단어 시퀀스의 평균 길이보다 넉넉하게 40으로 정의한다.

In [44]:
# 시퀀스 패딩 처리
max_len= 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding= 'post', maxlen= max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding= 'post', maxlen=max_len)

`sklearn.model_selection` 모듈의 `train_test_split()` 함수를 이용해 학습용과 테스트용 데이터셋을 8:2 비율로 분리한다. 

In [45]:
# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

이후 학습과 테스트용 출력 데이터(y_train, y_test)를 태그 사전 크기에 맞게 원-핫 인코딩한다.

In [46]:
# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train,num_classes = tag_size)
y_test = tf.keras.utils.to_categorical(y_test,num_classes = tag_size)

In [47]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", x_test.shape)
print("테스트 샘플 시퀀스 형상 : ", y_train.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (711, 40)
테스트 샘플 시퀀스 형상 :  (2844, 40, 8)
테스트 샘플 레이블 형상 :  (711, 40, 8)


In [49]:
# 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout,Bidirectional
from tensorflow.keras.optimizers import Adam

개체 인식 모델을 순차 모델 방식으로 구현한다. tag_size만큼의 출력 뉴런에서 제일 확률 높은 출력값 1개를 선택하는 문제이기 때문에 모델 출력 계층의 활성화 함수로 softmax를 사용하며 손실 함수로 categorical_crossentropy를 사용한다.

In [55]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim = 30, input_length=max_len, mask_zero= True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout = 0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size= 128, epochs =10)

Epoch 1/10
23/23 [==============================] - 27s 851ms/step - loss: 0.5029 - accuracy: 0.8416
Epoch 2/10
23/23 [==============================] - 20s 877ms/step - loss: 0.2357 - accuracy: 0.8961
Epoch 3/10
23/23 [==============================] - 21s 890ms/step - loss: 0.1510 - accuracy: 0.9278
Epoch 4/10
23/23 [==============================] - 21s 890ms/step - loss: 0.1109 - accuracy: 0.9468
Epoch 5/10
23/23 [==============================] - 20s 877ms/step - loss: 0.0857 - accuracy: 0.9616
Epoch 6/10
23/23 [==============================] - 20s 874ms/step - loss: 0.0618 - accuracy: 0.9733
Epoch 7/10
23/23 [==============================] - 20s 883ms/step - loss: 0.0475 - accuracy: 0.9796
Epoch 8/10
23/23 [==============================] - 20s 861ms/step - loss: 0.0387 - accuracy: 0.9830
Epoch 9/10
23/23 [==============================] - 21s 906ms/step - loss: 0.0314 - accuracy: 0.9861
Epoch 10/10
23/23 [==============================] - 19s 839ms/step - loss: 0.0283 - accura

In [56]:
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])

23/23 [==============================] - 2s 57ms/step - loss: 0.2110 - accuracy: 0.9371
평가 결과 :  0.9371342658996582


BIO 태그의 경우 실제 의미 있는 태그보다 의미 없는 O 태그가 대부분을 차지하고 있어 실제 우리가 원하는 성능과 무관하게 높은 점수로 계산한다. 따라서 개체명 인식에 사용되는 성능 평가는 F1 스코어를 계산하는 방법을 사용해야 한다.

In [70]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
  result = []
  for sequence in sequences:
    temp = []
    for pred in sequence: 
      pred_index = np.argmax(pred)
      temp.append(index_to_ner[pred_index].replace("PAD", "0"))
    result.append(temp)
  return result

In [71]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40 , 8)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

23/23 [==============================] - 1s 57ms/step


In [73]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=7421edaf731b4920d30d7f6acbc66d7a979ef13e66d843d787c3c86a4e4019f4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


F1 스코어를 계산하기 위해서는 정밀도와 재현율을 사용해야 하낟.

- 정확도 (Accuracy) : 실제 정답과 얼마나 유사한지 나타낸다.
- 정밀도 (Precision) : 정밀도가 높다고 해서 정확하다는 의미는 아니다. 정밀도가 높으면 결과값이 일정하게 분포되어 있는 것이다.
- 재현율 (Recall) : 실제 정답인 것들 중 예측 모델이 정답이라 예측한 것의 비율

F1 스코어란 정밀도와 재현율의 조화 평균을 의미한다.

위에서 predict() 함수를 통해 테스트용 데이터셋의 결과를 예측하고, 해당 함수의 입력으로 시퀀스 번호로 인코딩된 테스트용 단어 시퀀스(넘파이 배열)를 사용한다. 해당 함수의 결과로는 예측된 NER 태그 정보가 담긴 넘파이 배열이 반환된다.

seqeval.metrics 모듈의 classification_report() 함수를 통해 NER 태그별로 계싼된 정밀도와 재현율 그리고 F1 스코어를 출력한다. f1_score() 함수를 통해 F1 스코어만 불러올 수도 있다. 

In [75]:
# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWar

              precision    recall  f1-score   support

           _       0.60      0.53      0.56       644
         _DT       0.87      0.87      0.87       335
         _LC       0.76      0.51      0.61       312
         _OG       0.71      0.56      0.62       481
         _PS       0.81      0.41      0.55       374
         _TI       0.93      0.64      0.76        66

   micro avg       0.73      0.57      0.64      2212
   macro avg       0.78      0.59      0.66      2212
weighted avg       0.73      0.57      0.63      2212

F1-score: 63.7%


In [84]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
#new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.' .split()
new_sentence = '쟈니스 소속 가수인 아라시는 오늘 앨범을 발매한다.'.split()
new_x = []
for w in new_sentence:
  try:
    new_x.append(word_to_index.get(w,1))
  except KeyError:
    # 모르는 단어의 경우 OOV
    new_x.append(word_to_index['OOV'])

In [85]:
print("새로운 유형의 시퀀스 : ", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding="post", value=0, maxlen=max_len)

새로운 유형의 시퀀스 :  [1, 685, 1, 1, 286, 1, 1]


In [86]:
# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1) # 예측된 NER 인덱스값 추출

1/1 [==============================] - 0s 39ms/step


In [87]:
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)
for w, pred in zip(new_sentence, p[0]):
  print("{:10} {:5}".format(w, index_to_ner[pred]))

단어         예측된 NER
--------------------------------------------------
쟈니스        I    
소속         O    
가수인        O    
아라시는       I    
오늘         B_DT 
앨범을        I    
발매한다.      I    
